# algo 1

    for total_batches in range(total_n_pseudo_batches):
    
        z = tf.random.normal([batch_size, z_dim])
        pseudo_images = get_gen_images(z)
        teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images, mode=0)

        #generator training
        for ng in range(ng_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            generator_loss = generator_loss(teacher_logits, student_logits)

            #################################
            # BACK PROP AND tick schedulers #
            #################################  

        for ns in range(ns_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            student_loss = student_loss(teacher_logits, teacher_activations, 
                                        student_logits, student_activations, attn_beta)

            #################################
            # BACK PROP AND tick schedulers #
            #################################   

        ######################################################
        ### Val accuracy computation and best model saving ###
        ######################################################    

In [1]:
import tensorflow as tf
from net.generator import *
from utils.cosine_anealing import *
from utils.losses import *
from tensorflow.keras.optimizers import Adam
from net.wide_resnet import WideResidualNetwork
from train_scratch import *
import numpy as np

In [2]:
tf.__version__

'2.0.0'

In [3]:
z_dim = 100
batch_size = 128
ng_batches = 1
ns_batches = 5
attn_beta = 250
total_n_pseudo_batches = 3
n_generator_items = ng_batches + ns_batches
student_lr = 2e-3
generator_lr = 1e-3
number_of_batches = 3

teacher_model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0, output_activations=True)
teacher_model.load_weights('saved_models/cifar10_WRN-16-1_model.005.h5')

student_model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0, output_activations=True)
student_optimizer=Adam(learning_rate=student_lr)
student_scheduler = CosineAnnealingScheduler(T_max=number_of_batches, eta_max=student_lr, eta_min=0)

generator_model = generator(100)
generator_optimizer=Adam(learning_rate=generator_lr)
generator_scheduler = CosineAnnealingScheduler(T_max=number_of_batches, eta_max=generator_lr, eta_min=0)

student_model.trainable = True
teacher_model.trainable = False
generator_model.trainable = True

gen_loss_metric = tf.keras.metrics.Mean()
stu_loss_metric = tf.keras.metrics.Mean()

def cosine_lr_schedule(epoch, T_max, eta_max, eta_min=0):
    lr = eta_min + (eta_max - eta_min) * (1 + math.cos(math.pi * epoch / T_max)) / 2
    return lr

In [4]:
for total_batches in range(total_n_pseudo_batches):
    z = tf.random.normal([batch_size, z_dim])
    
    #generator training
    for ng in range(ng_batches):
        with tf.GradientTape() as gtape:
            pseudo_images = generator_model(z)
            teacher_logits, *teacher_activations = teacher_model(pseudo_images)
            student_logits, *student_activations = student_model(pseudo_images)
            generator_loss = kd_loss(tf.math.softmax(teacher_logits), tf.math.softmax(student_logits))

        gen_grads = gtape.gradient(generator_loss, generator_model.trainable_weights)
        
        #cosine annealing for learning rate
        generator_optimizer.learning_rate = cosine_lr_schedule(total_batches, total_n_pseudo_batches, generator_lr)
        
        #update gradient
        generator_optimizer.apply_gradients(zip(gen_grads, generator_model.trainable_weights))

        gen_loss_metric(generator_loss)

        print('step %s: generator mean loss = %s' % (total_batches, gen_loss_metric.result()))
    
    #student training
    for ns in range(ns_batches):
        with tf.GradientTape() as stape:
            pseudo_images = generator_model(z)
            teacher_logits, *teacher_activations = teacher_model(pseudo_images)
            student_logits, *student_activations = student_model(pseudo_images)
            std_loss = student_loss(teacher_logits, teacher_activations, 
                                student_logits, student_activations, attn_beta)

        st_grads = stape.gradient(std_loss, student_model.trainable_weights)
        
        student_optimizer.learning_rate = cosine_lr_schedule(total_batches, total_n_pseudo_batches, student_lr)
        student_optimizer.apply_gradients(zip(st_grads, student_model.trainable_weights))

        stu_loss_metric(std_loss)

        print('step %s-%s: studnt mean loss = %s' % (total_batches, ns, stu_loss_metric.result()))
    

step 0: generator mean loss = tf.Tensor(0.030972525, shape=(), dtype=float32)
step 0-0: studnt mean loss = tf.Tensor(0.754063, shape=(), dtype=float32)
step 0-1: studnt mean loss = tf.Tensor(0.69976777, shape=(), dtype=float32)
step 0-2: studnt mean loss = tf.Tensor(0.5874758, shape=(), dtype=float32)
step 0-3: studnt mean loss = tf.Tensor(0.6539779, shape=(), dtype=float32)
step 0-4: studnt mean loss = tf.Tensor(0.5643953, shape=(), dtype=float32)
step 1: generator mean loss = tf.Tensor(0.017535096, shape=(), dtype=float32)
step 1-0: studnt mean loss = tf.Tensor(0.5148973, shape=(), dtype=float32)
step 1-1: studnt mean loss = tf.Tensor(0.48786548, shape=(), dtype=float32)
step 1-2: studnt mean loss = tf.Tensor(0.46872106, shape=(), dtype=float32)
step 1-3: studnt mean loss = tf.Tensor(0.4505159, shape=(), dtype=float32)
step 1-4: studnt mean loss = tf.Tensor(0.42963776, shape=(), dtype=float32)
step 2: generator mean loss = tf.Tensor(0.012160477, shape=(), dtype=float32)
step 2-0: stu